# TP2 - Modeling using docplex

## 1. The `docplex` python package

`DOcplex` is a python package developped by IBM &mdash; It provides easy-to-use API for IBM solvers Cplex and Cpoptimizer.

DOcplex documentation for mathematical programming can be found here: http://ibmdecisionoptimization.github.io/docplex-doc/#mathematical-programming-modeling-for-python-using-docplex-mp-docplex-mp

## 2. Solving TSP using `docplex`

### 2.1. TSP model using `docplex`

**Exercice:** Using `docplex`, create a model for the travelling salesman problem using the MTZ or Flow formulation and compare them.

In [2]:
#!pip install cplex

In [1]:
from docplex.mp.model import Model
import tsp.data as data
import numpy as np


#distances = data.grid42
distances = data.grid17
N = len(distances)

tsp_flow = Model("TSP Flow")
tsp_flow.log_output = False

tsp_mtz = Model("TSP MTZ")
tsp_mtz.log_output = True

#Variables
x = [tsp_flow.binary_var_list(N,name=f'x{i}_') for i in range(N)]
y = [tsp_flow.integer_var_list(N,name=f'y{i}_') for i in range(N)]

t = [tsp_mtz.binary_var_list(N,name=f't{i}_') for i in range(N)]
u = tsp_mtz.integer_var_list(N,name=f'u_')

#Objective
#tsp.minimize(sum(tsp.dot(distances[i][j], x[i][j]) for i in range(N) for j in range(N)))
mySum = 0
for i in range(N):
    for j in range(N):
        mySum += distances[i][j]*x[i][j]
tsp_flow.minimize(mySum)

mySum = 0
for i in range(N):
    for j in range(N):
        mySum += distances[i][j]*t[i][j]
tsp_mtz.minimize(mySum)

#Constraints
#Constraints on variable x & t
for i in range(N) :
    tsp_flow.add_constraint(sum(x[i][j] for j in range(N) if i!=j) == 1)
    tsp_mtz.add_constraint(sum(t[i][j] for j in range(N) if i!=j) == 1)
    
for j in range(N) :
    tsp_flow.add_constraint(sum(x[i][j] for i in range(N) if i!=j) == 1)
    tsp_mtz.add_constraint(sum(t[i][j] for i in range(N) if i!=j) == 1)
    
for i in range(N):
    tsp_flow.add_constraint(x[i][i] == 0)
    tsp_mtz.add_constraint(t[i][i] == 0)

#Constraints on variable u    
tsp_mtz.add_constraint(u[0] == 1)

for i in range(1,N) : 
    tsp_mtz.add_constraint(2 <= u[i])
    tsp_mtz.add_constraint(u[i] <= N)
    tsp_mtz.add_constraints( [u[i]- u[j] + 1 <= (N-1)*(1-t[i][j]) for j in range(1,N)] )

#Constraints on variable y    
tsp_flow.add_constraint(sum(y[0][j] for j in range(N)) == 1)

for i in range(1,N):
    tsp_flow.add_constraint(sum(y[i][j] for j in range(N)) == sum(y[j][i] for j in range(N))+ 1)

for i in range(N):
    tsp_flow.add_constraints([y[i][j] <= (N * x[i][j]) for j in range(N)])


solution_flow = tsp_flow.solve()
print("Flow : z* =", solution_flow.objective_value)


#solution_mtz = tsp_mtz.solve()
#print("MTZ  : z* =", solution_mtz.objective_value)

Flow : z* = 2085.0


The largest set of distances contains 42 nodes, and should be easily solved by `docplex`.

### 2.2. Generating random TSP instances

**Question:** What method could you implement to generate a realistic set of distances for $n$ customers?

**Exercice:** Implement the method proposed above and test it.

In [2]:
import numpy as np
import scipy.spatial.distance as sc

def generate_distances(n: int):
    coords = np.random.rand(n, 2) #2 est le nombre de dimensions du pb
    return sc.cdist(coords, coords, metric='euclidean') #librairie qui permet d'avoir une matrice symétrique


from docplex.mp.model import Model

distances = generate_distances(20)
#distances = generate_distances(50)
print(distances)

N = len(distances)

tsp = Model("TSP")
tsp.log_output = True

# TODO: Copy your model from the first question here.
#Variables
x = [tsp.binary_var_list(N,name=f'x{i}_') for i in range(N)]
y = [tsp.integer_var_list(N,name=f'y{i}_') for i in range(N)]

#Objective
#tsp.minimize(sum(tsp.dot(distances[i][j], x[i][j]) for i in range(N) for j in range(N)))
mySum = 0
for i in range(N):
    for j in range(N):
        mySum += distances[i][j]*x[i][j]
tsp.minimize(mySum)

#Constraints
#Constraints on variable x & t
for i in range(N) :
    tsp.add_constraint(sum(x[i][j] for j in range(N) if i!=j) == 1)
    
for j in range(N) :
    tsp.add_constraint(sum(x[i][j] for i in range(N) if i!=j) == 1)
    
for i in range(N):
    tsp.add_constraint(x[i][i] == 0)

#Constraints on variable y    
tsp.add_constraint(sum(y[0][j] for j in range(N)) == 1)

for i in range(1,N):
    tsp.add_constraint(sum(y[i][j] for j in range(N)) == sum(y[j][i] for j in range(N))+ 1)

for i in range(N):
    tsp.add_constraints([y[i][j] <= (N * x[i][j]) for j in range(N)])

solution = tsp.solve()
print("z* =", solution.objective_value)

[[0.         0.72798484 0.92590762 0.92871737 0.37500214 0.02633906
  0.51203369 0.89532182 0.68366722 0.43255403 0.12448558 0.88196004
  0.58069398 0.82143299 0.76935641 0.8775191  1.03080405 0.45313714
  0.79148442 0.92999528]
 [0.72798484 0.         0.2414499  0.56610937 0.77353513 0.70189247
  0.32028937 0.45961029 0.15181152 0.40487926 0.81500654 0.2348652
  0.80067825 0.44194375 0.38543712 0.29240397 0.94771109 0.94428735
  0.73522026 0.65080475]
 [0.92590762 0.2414499  0.         0.44696315 0.8898497  0.90080367
  0.44091384 0.33457063 0.24407444 0.53533581 0.99624425 0.06509493
  0.8585598  0.35985828 0.33238685 0.14099484 0.87476171 1.05919241
  0.70967408 0.55040447]
 [0.92871737 0.56610937 0.44696315 0.         0.71363886 0.91036912
  0.45492528 0.11328519 0.43133557 0.50601256 0.94429481 0.39218567
  0.58103493 0.13263129 0.19724241 0.3094841  0.43386258 0.85301717
  0.32976192 0.10719624]
 [0.37500214 0.77353513 0.8898497  0.71363886 0.         0.37597309
  0.4594903  0.72

## 3. Solving Warehouse Allocation using Benders decomposition with `docplex`

### 3.1. The warehouse problem

A company needs to supply a set of $n$ clients and needs to open new warehouses (from a
set of $m$ possible warehouses).
Opening a warehouse $j$ costs $f_j$ and supplying a client $i$ from a warehouse $j$ costs $c_{ij}$ per supply unit.
Which warehouses should be opened in order to satisfy all clients while minimizing the total cost?

### 3.2. Solving the warehouse problem with a single ILP

- $y_{j} = 1$ if and only if warehouse $j$ is opened.
- $x_{ij}$ is the fraction supplied from warehouse $j$ to customer $i$.

$
\begin{align}
  \text{min.} \quad & \sum_{i=1}^{n} \sum_{j=1}^{m} c_{ij} x_{ij} + \sum_{j=1}^{m} f_{j} y_{j} & \\
  \text{s.t.} \quad & \sum_{j=1}^{m} x_{ij} = 1, & \forall i \in\{1,\ldots,n\}\\
                    & x_{ij} \leq y_{j}, & \forall i\in\{1,\ldots,n\},\forall j\in\{1,\ldots,m\}\\
                    & y_{j} \in \left\{0,~1\right\}, & \forall j \in \left\{1,~\ldots,~m\right\}\\
                    & x_{ij} \geq 0, & \forall i \in \left\{1,~\ldots,~n\right\}, \forall j \in \left\{1,~\ldots,~m\right\}
\end{align}
$


**Exercice:** Implement the ILP model for the warehouse allocation problem and test it on the given instance.

In [41]:
from docplex.mp.model import Model

# We will start with a small instances with 3 warehouses and 3 clients:
N = 3
M = 3

# Opening and distribution costs:
f = [20, 20, 20]
c = [[15, 1, 2], [1, 16, 3], [4, 1, 17]]

wa = Model("Warehouse Allocation")
wa.log_output = True

# TODO: Model for the warehouse allocation.

#Variables
x = [wa.integer_var_list(M,name=f'x{i}_') for i in range(N)]
y = wa.binary_var_list(M,name=f'y_')

#Objective
wa.minimize(sum(c[i][j]*x[i][j] for i in range(N) for j in range(M)) + sum(f[j]*y[j] for j in range(M)))

#Constraints
for i in range(N):
    wa.add_constraint(sum(x[i][j] for j in range(M)) == 1)

for i in range(N):
    for j in range(M):
        wa.add_constraint(x[i][j] <= y[j])
    
"""for j in range(M):
    wa.add_constraint(y[j] >= 0)
    wa.add_constraint(y[i] <= 1)"""
#inutile car y est une liste binaire
    
for i in range (N):
    for j in range(M):
        wa.add_constraint(x[i][j] >= 0)

solution = wa.solve()
print("z* =", solution.objective_value)

CPXPARAM_Read_DataCheck                          1
Found incumbent of value 40.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 9 rows and 0 columns.
Reduced MIP has 12 rows, 12 columns, and 27 nonzeros.
Reduced MIP has 12 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
Reduced MIP has 12 rows, 12 columns, and 27 nonzeros.
Reduced MIP has 12 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Clique table members: 12.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 32 threads.
Root relaxation solution time = 0.00 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                   

### 3.3. Benders' decomposition for the Warehouse Allocation problem

We are going to use Benders' decomposition to solve the Warehouse Allocation problem. 

#### Dual subproblem

$
\begin{align*}
\text{max.} \quad & \sum_{i=1}^{n} v_{i} - \sum_{i=1}^{n}\sum_{j=1}^{m} \bar{y}_{j} u_{ij} & \\
\text{s.t.} \quad & v_{i} - u_{ij} \leq c_{ij}, & \forall i\in\{1,\ldots,n\},\forall j\in\{1,\ldots,m\}\\
                  & v_{i} \in\mathbb{R},\ u_{ij} \geq 0 & \forall i \in\{1,\ldots,n\}, \forall j\in\{1,\ldots,m\}
\end{align*}
$

#### Master problem

$
\begin{align*}
  \text{min.} \quad & \sum_{j=1}^{m} f_j y_j + z & \\
  \text{s.t.} \quad & z \geq \sum_{i=1}^{n}v_i^p - \sum_{i=1}^{n} \sum_{j=1}^{m} u_{ij}^p y_j, & \forall p\in l_1\\
                    & 0 \geq \sum_{i=1}^{n}v_i^r - \sum_{i=1}^{n} \sum_{j=1}^{n} u_{ij}^r y_j, & \forall r\in l_2\\
                    & y_{j} \in\{0,1\}, & \forall j\in\{1,\ldots,m\}
\end{align*}
$

**Exercice:** Implement the method `create_master_problem` that creates the initial master problem (without feasibility or optimality constraints) for the warehouse allocation problem.

<div class="alert alert-info alert-block">

You can use `print(m.export_as_lp_string())` to display a textual representation of a `docplex` model `m`.
    
</div>

In [ ]:
from docplex.mp.model import Model
from docplex.mp.linear import Var
from docplex.mp.constr import AbstractConstraint
from typing import List, Sequence, Tuple


def create_master_problem(
    N: int, M: int, f: Sequence[float], c: Sequence[Sequence[float]]
) -> Tuple[Model, Var, Sequence[Var]]:
    """
    Creates the initial Benders master problem for the Warehouse Allocation problem.

    Args:
        N: Number of clients.
        M: Number of warehouses.
        f: Array-like containing costs of opening warehouses.
        c: 2D-array like containing transport costs from client to warehouses.

    Returns:
        A 3-tuple containing the docplex problem, the z variable and the y variables.
    """

    wa = Model("Warehouse Allocation - Benders master problem")

    #Master problem
    #variables
    u = [wa.integer_var_list(M,name=f'u{i}_') for i in range(N)]
    v = wa.integer_var_list(N,name=f'v_')
    y = wa.binary_var_list(M,name=f'y_')
    
    #Objective
    

    #Constraints
    wa.add_constraint((sum(v[i]) for i in range(N)-sum(u[i][j]*y[j]) for i in range(N) for j in range(M)) <= z)
    wa


    return wa, z, y


# Check your method:
wa, z, y = create_master_problem(N, M, f, c)
print(wa.export_as_lp_string())

**Exercice:** Implement the method `add_optimality_constraints` that add optimality constraints to the Benders master problem. 

In [ ]:
def add_optimality_constraint(
    N: int,
    M: int,
    wa: Model,
    z: Var,
    y: Sequence[Var],
    v: Sequence[float],
    u: Sequence[Sequence[float]],
) -> List[AbstractConstraint]:
    """
    Adds an optimality constraints to the given Warehouse Allocation model
    using the given optimal values from the Benders dual subproblem.

    Args:
        N: Number of clients.
        M: Number of warehouses.
        wa: The Benders master problem (docplex.mp.model.Model).
        z: The z variable of the master problem.
        y: The y variables of the master problem.
        v: The optimal values for the v variables of the Benders dual subproblem.
        u: The optimal values for the u variables of the Benders dual subproblem.

    Return: The optimality constraint added.
    """
    return ...  # TODO

**Exercice:** Implement the method `add_feasibility_constraints` that add feasibility constraints to the Benders master problem. 

In [ ]:
def add_feasibility_constraints(
    N: int,
    M: int,
    wa: Model,
    z: Var,
    y: Sequence[Var],
    v: Sequence[float],
    u: Sequence[Sequence[float]],
) -> List[AbstractConstraint]:
    """
    Adds an optimality constraints to the given Warehouse Allocation model
    using the given optimal values from the Benders dual subproblem.

    Args:
      - N: Number of clients.
      - M: Number of warehouses.
      - wa: The Benders master problem (docplex.mp.model.Model).
      - z: The z variable of the master problem.
      - y: The y variables of the master problem.
      - v: The extreme rays for the v variables of the Benders dual subproblem.
      - u: The extreme rays for the u variables of the Benders dual subproblem.

    Returns:
        The feasibility constraint added.
    """
    # TODO:
    return

**Exercice:** Implement the method `create_dual_subproblem` that, given a solution `y` of the master problem, create the corresponding Benders dual subproblem.

$
\begin{align*}
\text{max.} \quad & \sum_{i=1}^{n} v_{i} - \sum_{i=1}^{n}\sum_{j=1}^{m} \bar{y}_{j} u_{ij} & \\
\text{s.t.} \quad & v_{i} - u_{ij} \leq c_{ij}, & \forall i\in\{1,\ldots,n\},\forall j\in\{1,\ldots,m\}\\
                  & v_{i} \in\mathbb{R},\ u_{ij} \geq 0 & \forall i \in\{1,\ldots,n\}, \forall j\in\{1,\ldots,m\}
\end{align*}
$

In [ ]:
from docplex.mp.model import Model


def create_dual_subproblem(
    N: int, M: int, f: Sequence[float], c: Sequence[Sequence[float]], y: Sequence[int]
) -> Tuple[Model, Sequence[Var], Sequence[Sequence[Var]]]:
    """
    Creates a Benders dual subproblem for the Warehouse Allocation problem corresponding
    to the given master solution.

    Args:
        N: Number of clients.
        M: Number of warehouses.
        f: Array-like containing costs of opening warehouses.
        c: 2D-array like containing transport costs from client to warehouses.
        y: Values of the y variables from the Benders master problem.

    Returns:
        A 3-tuple containing the docplex problem, the v variable and the u variables.
    """

    dsp = Model("Warehouse Allocation - Benders dual subproblem")

    # We disable pre-solve to be able to retrieve a meaningful status in the main
    # algorithm:
    dsp.parameters.preprocessing.presolve.set(0)

    ...  # TODO

    return dsp, v, u


# Check your method (assuming y = [1 1 1 ... 1]):
dsp, v, u = create_dual_subproblem(N, M, f, c, [1] * M)
print(dsp.export_as_lp_string())

**Exercice:** Using the methods you implemented, write the Benders decomposition algorithm for the warehouse allocation problem.

<div class="alert alert-block alert-info">

The `get_extreme_rays` function can be used to retrieve the extreme rays associated with an unbounded solution of the dual subproblem.
    
</div>

<div class="alert alert-block alert-info">
    
You can use `model.get_solve_status()` to obtain the status of the resolution and compare it to members of `JobSolveStatus`:
    
```python
if model.get_solve_status() == JobSolveStatus.OPTIMAL_SOLUTION:
    pass
```
    
</div>

In [ ]:
from docplex.mp.model import Model
from docplex.util.status import JobSolveStatus


def get_extreme_rays(
    N: int, M: int, model: Model, v: Sequence[Var], u: Sequence[Sequence[Var]]
) -> Tuple[Sequence[float], Sequence[Sequence[float]]]:
    """
    Retrieves the extreme rays associated to the dual subproblem.

    Args:
        N: Number of clients.
        M: Number of warehouses.
        model: The Benders dual subproblem model (docplex.mp.model.Model).
        v: 1D array containing the v variables of the subproblem.
        u: Either a 2D array of a tuple-index dictionary containing the u variables
            of the subproblem.

    Returns:
        A 2-tuple containing the list of extreme rays correspondig to v,
        and the 2D-list of extreme rays corresponding to u.
    """
    ray = model.get_engine().get_cplex().solution.advanced.get_ray()

    if isinstance(u, dict):

        def get_uij(i, j):
            return u[i, j]

    else:

        def get_uij(i, j):
            return u[i][j]

    return (
        [ray[v[i].index] for i in range(N)],
        [[ray[get_uij(i, j).index] for j in range(M)] for i in range(N)],
    )


# We will start with a small instances with 3 warehouses and 3 clients:
N = 3
M = 3

# Opening and distribution costs:
f = [20, 20, 20]
c = [[15, 1, 2], [1, 16, 3], [4, 1, 17]]

# We stop iterating if the new solution is less than epsilon
# better than the previous one:
epsilon = 1e-6

wa, z, y = create_master_problem(N, M, f, c)

n = 0
while True:

    # Print iteration:
    n = n + 1
    print("Iteration {}".format(n))

    ...  # TODO

print("Done.")

### 3.4. Generating instances for the Warehouse Allocation problem

**Exercice:** Using the TSP instances contained in `tsp.data` or the `generate_distances` method, create instances for the warehouse allocation problem with randomized opening costs.

<div class="alert alert-block alert-danger"></div>